<a href="https://colab.research.google.com/github/gowripreetham/SJSU_AutoGluon_Assignment/blob/main/Copy_of_tabular_multimodal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# AutoGluon - Multimodal Tabular Example (Text + Numeric)
# Part: B.i.2 of AutoGluon Assignment
# ============================================================

!pip -q install -U pip
!pip -q install -U autogluon
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
from sklearn.model_selection import train_test_split

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.8.0+cu126 requires torch==2.8.0, but you have torch 2.7.1 which is incompatible.


In [ ]:
# ============================================================
# 1️⃣ Load dataset with both text + numeric columns
# ============================================================

# Using the Amazon product reviews dataset (lightweight text + numeric)
data_url = "https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv"
data = pd.read_csv(data_url)

print("✅ Dataset loaded!")
print("Columns:", list(data.columns))
print(data.head())

# Keep only useful columns (the dataset has many)
cols_to_keep = [
    'age', 'education-num', 'occupation', 'hours-per-week',
    'marital-status', 'relationship', 'race', 'sex', 'class', 'native-country'
]
data = data[cols_to_keep]

# Rename label column (it's called 'class' here, not 'income')
data = data.rename(columns={'class': 'label'})

# Add a fake text column to simulate product description
data['text_description'] = (
    "This person works as " + data['occupation'].astype(str) +
    " and is " + data['relationship'].astype(str)
)

print("\n✅ Cleaned dataset:")
data.head()

✅ Dataset loaded!
Columns: ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'class']
   age   workclass  fnlwgt   education  education-num       marital-status  \
0   25     Private  178478   Bachelors             13        Never-married   
1   23   State-gov   61743     5th-6th              3        Never-married   
2   46     Private  376789     HS-grad              9        Never-married   
3   55           ?  200235     HS-grad              9   Married-civ-spouse   
4   36     Private  224541     7th-8th              4   Married-civ-spouse   

           occupation    relationship    race      sex  capital-gain  \
0        Tech-support       Own-child   White   Female             0   
1    Transport-moving   Not-in-family   White     Male             0   
2       Other-service   Not-in-family   White     Male             0   
3              

,age,education-num,occupation,hours-per-week,marital-status,relationship,race,sex,label,native-country,text_description
0,25,13,Tech-support,40,Never-married,Own-child,White,Female,<=50K,United-States,This person works as Tech-support and is Own...
1,23,3,Transport-moving,35,Never-married,Not-in-family,White,Male,<=50K,United-States,This person works as Transport-moving and is ...
2,46,9,Other-service,15,Never-married,Not-in-family,White,Male,<=50K,United-States,This person works as Other-service and is No...
3,55,9,?,50,Married-civ-spouse,Husband,White,Male,>50K,United-States,This person works as ? and is Husband
4,36,4,Handlers-cleaners,40,Married-civ-spouse,Husband,White,Male,<=50K,El-Salvador,This person works as Handlers-cleaners and is...


In [ ]:
# ============================================================
# 2️⃣ Train/Test split
# ============================================================
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
label = 'label'

print("\n🎯 Target column:", label)
print("Training data shape:", train_data.shape)


🎯 Target column: label
Training data shape: (31258, 11)


In [ ]:
# ============================================================
# 3️⃣ Train AutoGluon multimodal model
# ============================================================
# AutoGluon automatically detects numeric, categorical, and text features

predictor = TabularPredictor(
    label=label,
    problem_type='binary',
    eval_metric='accuracy',
    path='AutogluonModels_Multimodal/'
).fit(
    train_data,
    time_limit=120,  # runs quickly on CPU
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Memory Avail:       11.33 GB / 12.67 GB (89.4%)
Disk Space Avail:   61.60 GB / 107.72 GB (57.2%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU memory.
	presets='best'    : Maximize accuracy. Recommended for most users. Use in competitions and

In [ ]:
# ============================================================
# 4️⃣ Evaluate on Test Data
# ============================================================
print("\n🔍 Evaluating model on test data...")
results = predictor.evaluate(test_data, silent=True)
print("📊 Evaluation Results:", results)



🔍 Evaluating model on test data...
📊 Evaluation Results: {'accuracy': 0.835828534868842, 'balanced_accuracy': np.float64(0.7452976128277131), 'mcc': np.float64(0.5172528621570761), 'roc_auc': np.float64(0.8846631308113503), 'f1': 0.618948618948619, 'precision': 0.6675208199871877, 'recall': 0.5769656699889258}


In [ ]:
# ============================================================
# 5️⃣ Leaderboard
# ============================================================
print("\n🏆 Model Leaderboard:")
leaderboard = predictor.leaderboard(test_data, silent=True)
leaderboard


🏆 Model Leaderboard:


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,0.837492,0.8408,accuracy,0.154515,0.040694,13.752096,0.154515,0.040694,13.752096,1,True,1
1,LightGBM,0.835829,0.8408,accuracy,0.044019,0.013122,1.065199,0.044019,0.013122,1.065199,1,True,2
2,XGBoost,0.835829,0.8420,accuracy,0.131860,0.046275,5.305723,0.131860,0.046275,5.305723,1,True,9
3,WeightedEnsemble_L2,0.835829,0.8420,accuracy,0.134494,0.047685,5.442991,0.002634,0.001410,0.137267,2,True,10
4,NeuralNetFastAI,0.833525,0.8364,accuracy,0.196426,0.057930,30.156184,0.196426,0.057930,30.156184,1,True,8
5,CatBoost,0.833013,0.8388,accuracy,0.027709,0.006658,10.734088,0.027709,0.006658,10.734088,1,True,5
6,RandomForestGini,0.814459,0.8128,accuracy,1.879256,0.225133,11.771107,1.879256,0.225133,11.771107,1,True,3
7,RandomForestEntr,0.813180,0.8132,accuracy,1.151094,0.245291,11.584436,1.151094,0.245291,11.584436,1,True,4
8,ExtraTreesGini,0.813052,0.8168,accuracy,0.791203,0.416653,8.702054,0.791203,0.416653,8.702054,1,True,6
9,ExtraTreesEntr,0.812412,0.8120,accuracy,1.162918,0.240547,8.637492,1.162918,0.240547,8.637492,1,True,7


In [ ]:
# ============================================================
# 6️⃣ Feature Importance (optional)
# ============================================================
print("\n📈 Feature Importance:")
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 10 features using 5000 rows with 5 shuffle sets...



📈 Feature Importance:


	17.87s	= Expected runtime (3.57s per shuffle set)
	17.26s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
education-num,0.03908,0.002552,0.000002,5,0.044334,0.033826
marital-status,0.03800,0.005320,0.000045,5,0.048953,0.027047
age,0.02060,0.005144,0.000430,5,0.031191,0.010009
text_description,0.01432,0.003100,0.000248,5,0.020704,0.007936
hours-per-week,0.01344,0.003251,0.000381,5,0.020134,0.006746
occupation,0.01120,0.003418,0.000923,5,0.018237,0.004163
sex,0.00208,0.001494,0.017881,5,0.005156,-0.000996
native-country,0.00160,0.002088,0.080897,5,0.005899,-0.002699
relationship,0.00152,0.001579,0.048823,5,0.004770,-0.001730
race,0.00064,0.000555,0.030708,5,0.001783,-0.000503


In [ ]:
# ============================================================
# 7️⃣ Summary
# ============================================================
print("""
✅ SUCCESS! You have demonstrated a Multimodal Tabular model using AutoGluon.
   - Dataset included numeric, categorical, and text columns.
   - AutoGluon automatically processed mixed feature types.
   - Model trained and evaluated successfully.
   - Leaderboard shows best performing models.

Save this notebook (with outputs) to:
colabs/multimodal_tabular/tabular-multimodal.ipynb
""")


✅ SUCCESS! You have demonstrated a Multimodal Tabular model using AutoGluon.
   - Dataset included numeric, categorical, and text columns.
   - AutoGluon automatically processed mixed feature types.
   - Model trained and evaluated successfully.
   - Leaderboard shows best performing models.

Save this notebook (with outputs) to:
colabs/multimodal_tabular/tabular-multimodal.ipynb

